<a href="https://colab.research.google.com/github/ali-talebi/norouz1404/blob/main/lstm_thesis_2025_09_30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/Malek_Master/code/DATA_PICTURE_3D_Change.zip

Archive:  /content/drive/MyDrive/Malek_Master/code/DATA_PICTURE_3D_Change.zip
   creating: DATA_PICTURE_3D_Change/
   creating: DATA_PICTURE_3D_Change/picture_alpha/
  inflating: DATA_PICTURE_3D_Change/picture_alpha/alpha_0.001_void_0.1.csv  
  inflating: DATA_PICTURE_3D_Change/picture_alpha/alpha_0.001_void_0.2.csv  
  inflating: DATA_PICTURE_3D_Change/picture_alpha/alpha_0.001_void_0.3.csv  
  inflating: DATA_PICTURE_3D_Change/picture_alpha/alpha_0.001_void_0.4.csv  
  inflating: DATA_PICTURE_3D_Change/picture_alpha/alpha_0.002_void_0.1.csv  
  inflating: DATA_PICTURE_3D_Change/picture_alpha/alpha_0.002_void_0.2.csv  
  inflating: DATA_PICTURE_3D_Change/picture_alpha/alpha_0.002_void_0.3.csv  
  inflating: DATA_PICTURE_3D_Change/picture_alpha/alpha_0.002_void_0.4.csv  
  inflating: DATA_PICTURE_3D_Change/picture_alpha/alpha_0.003_void_0.1.csv  
  inflating: DATA_PICTURE_3D_Change/picture_alpha/alpha_0.003_void_0.2.csv  
  inflating: DATA_PICTURE_3D_Change/picture_alpha/alpha_0.003_vo

In [ ]:
!pip install visualkeras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.8 MB/s eta 0:00:00


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential , Model
from keras.layers import Dense , Dropout , Conv2D , BatchNormalization , Flatten , Conv2DTranspose , Input
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model
import visualkeras
from sklearn.model_selection import train_test_split

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
path_data = "/content/DATA_PICTURE_3D_Change"

store_data_set = {
					'picture_alpha':[] ,
					'picture_health' : [] ,
					'picture_void' : [] ,
					'picture_void_alpha' : []
				}

def function_conver_and_load_image() :
  for path in os.listdir(path_data) :
    print(path)
    cursur = None
    if path == 'picture_alpha' :
      cursur = 'picture_alpha'
    elif path == 'picture_health' :
      cursur = 'picture_health'
    elif path == 'picture_void' :
      cursur = 'picture_void'
    elif path == 'picture_void_alpha' :
      cursur = 'picture_void_alpha'

    for file in os.listdir(f'{path_data}/{path}') :
      combined_root = f'{path_data}/{path}/{file}'
      print(combined_root)
      df = pd.read_csv(combined_root)
      matrix_picture_z = []
      matrix_picture_y = []
      matrix_picture_x = []
      for index in range(len(df)) :
        value_z = df.iloc[index , 2 ]
        matrix_picture_z.append(value_z)
        value_y = df.iloc[index , 1 ]
        matrix_picture_y.append(value_y)
        value_x = df.iloc[index , 0 ]
        matrix_picture_x.append(value_x)

      matrix_picture_z = np.array(matrix_picture_z).reshape(16,3)
      matrix_picture_y = np.array(matrix_picture_y).reshape(16,3)
      matrix_picture_x = np.array(matrix_picture_x).reshape(16,3)

      matrix_picture = {'x':matrix_picture_x , 'y':matrix_picture_y , 'z':matrix_picture_z }
      store_data_set[cursur].append(matrix_picture)


  return store_data_set
store_data_set = function_conver_and_load_image()


picture_void
/content/DATA_PICTURE_3D_Change/picture_void/alpha_0.004_void_0.2.csv
/content/DATA_PICTURE_3D_Change/picture_void/alpha_0.089_void_0.2.csv
/content/DATA_PICTURE_3D_Change/picture_void/alpha_0.076_void_0.3.csv
/content/DATA_PICTURE_3D_Change/picture_void/alpha_0.048_void_0.1.csv
/content/DATA_PICTURE_3D_Change/picture_void/alpha_0.081_void_0.1.csv
/content/DATA_PICTURE_3D_Change/picture_void/alpha_0.097_void_0.4.csv
/content/DATA_PICTURE_3D_Change/picture_void/alpha_0.019_void_0.3.csv
/content/DATA_PICTURE_3D_Change/picture_void/alpha_0.003_void_0.2.csv
/content/DATA_PICTURE_3D_Change/picture_void/alpha_0.057_void_0.3.csv
/content/DATA_PICTURE_3D_Change/picture_void/alpha_0.032_void_0.2.csv
/content/DATA_PICTURE_3D_Change/picture_void/alpha_0.023_void_0.4.csv
/content/DATA_PICTURE_3D_Change/picture_void/alpha_0.09_void_0.1.csv
/content/DATA_PICTURE_3D_Change/picture_void/alpha_0.028_void_0.2.csv
/content/DATA_PICTURE_3D_Change/picture_void/alpha_0.092_void_0.2.csv
/content

In [ ]:
total_health_x_y_z , label_health = [] , []
for i in store_data_set :
  for img in store_data_set[i] :
    if i == 'picture_health' :
      total_health_x_y_z.append(img)
      label_health.append(i)


x_health , y_health , z_health = [] , [] , []

for i in total_health_x_y_z:
  x_health.append(i['x'])
  y_health.append(i['y'])
  z_health.append(i['z'])


x_health = np.array(x_health).reshape(-1 , 16 , 3 , 1 )
y_health = np.array(y_health).reshape(-1 , 16 , 3 , 1 )
z_health = np.array(z_health).reshape(-1 , 16 , 3 , 1 )
z_health = np.where(z_health < 0 , 0 , z_health )

TOTAL_X_health_vector = []
TOTAL_Target_X_health_vector = []

window_size = 2
for picture in z_health:
    picture_reshaped = picture.reshape(16, 3)  # 16 ردیف، هر ردیف 3 ستون (x,y,z)
    n_samples = picture_reshaped.shape[0] - window_size
    for i in range(n_samples):
        # ورودی = 3 ردیف پشت سر هم
        TOTAL_X_health_vector.append(picture_reshaped[i : i + window_size])   # (3,3)
        # خروجی = ردیف بعد از این 3 تایی
        TOTAL_Target_X_health_vector.append(picture_reshaped[i + window_size]) # (3,)

TOTAL_X_health_vector_array = np.array(TOTAL_X_health_vector).reshape(-1, window_size, 3)
TOTAL_Target_X_health_vector_array = np.array(TOTAL_Target_X_health_vector).reshape(-1, 3)

print(TOTAL_X_health_vector_array.shape)   # (N, 3, 3)
print(TOTAL_Target_X_health_vector_array.shape)  # (N, 3)

(5544, 2, 3)
(5544, 3)


In [ ]:
reverse_total_health_x_y_z , reverse_label_health = [] , []
for i in store_data_set :
  for img in store_data_set[i] :
    if i == 'picture_health' :
      reverse_total_health_x_y_z.append(img)
      reverse_label_health.append(i)


rev_x_health , rev_y_health , rev_z_health = [] , [] , []

for i in reverse_total_health_x_y_z:
  rev_x_health.append(i['x'])
  rev_y_health.append(i['y'])
  rev_z_health.append(i['z'])


rev_x_health = np.array(rev_x_health).reshape(-1 , 16 , 3 , 1 )
rev_y_health = np.array(rev_y_health).reshape(-1 , 16 , 3 , 1 )
rev_z_health = np.array(rev_z_health).reshape(-1 , 16 , 3 , 1 )
rev_z_health = np.where(rev_z_health < 0 , 0 , rev_z_health )

rev_TOTAL_X_health_vector = []
rev_TOTAL_Target_X_health_vector = []

window_size = 2
for picture in rev_z_health:
    picture_reshaped = picture.reshape(16, 3)  # 16 ردیف، هر ردیف 3 ستون (x,y,z)
    n_samples = picture_reshaped.shape[0] - window_size
    for i in range(n_samples,0,-1):
        # ورودی = 3 ردیف پشت سر هم
        rev_TOTAL_X_health_vector.append(picture_reshaped[-(i + window_size):-i])   # (3,3)
        # خروجی = ردیف بعد از این 3 تایی
        rev_TOTAL_Target_X_health_vector.append(picture_reshaped[-i]) # (3,)

rev_TOTAL_X_health_vector = np.array(rev_TOTAL_X_health_vector).reshape(-1, window_size, 3)
rev_TOTAL_Target_X_health_vector = np.array(rev_TOTAL_Target_X_health_vector).reshape(-1, 3)

print(rev_TOTAL_X_health_vector.shape)   # (N, 3, 3)
print(rev_TOTAL_Target_X_health_vector.shape)  # (N, 3)

(5544, 2, 3)
(5544, 3)


In [ ]:
x_train , x_test , y_train , y_test = train_test_split(TOTAL_X_health_vector_array,TOTAL_Target_X_health_vector_array , test_size=0.3 , random_state = 42 )

In [ ]:
rev_x_train , rev_x_test , rev_y_train , rev_y_test = train_test_split(rev_TOTAL_X_health_vector,rev_TOTAL_Target_X_health_vector , test_size=0.3 , random_state = 42 )

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# ————— تعریف مدل —————
model_forward = Sequential([
    # لایه LSTM: 4 واحد پنهان، فقط خروجی نهایی را باز می‌گرداند
    LSTM(64, input_shape=(2, 3), return_sequences=False),
    # برای کاهش بیش‌برازش، کمی Dropout
    Dropout(0.2),
    # یک لایه میانی برای افزایش ظرفیت غیرخطی مدل
    Dense(16, activation='relu'),
    # لایه خروجی با 3 نورون (برای هر feature یک مقدار)
    Dense(3)
])

model_forward.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mse']
)
model_forward.summary()

# ————— تنظیم EarlyStopping —————
es = EarlyStopping(
    monitor='val_loss',
    patience=50,
    restore_best_weights=True
)

# ————— آموزش مدل —————
history = model_forward.fit(
    x_train, y_train,
    epochs=50,
    batch_size=64,
    validation_split=0.2,
    # callbacks=[es],
    verbose=2
)

# ————— پیش‌بینی —————
# مثال: پیش‌بینی برای آخرین دو سطر
x_new = x_train[-1][np.newaxis, ...]      # شکل (1,2,3)
y_pred = model_forward.predict(x_new)       # شکل (1,3)
print("پیش‌بینی:", y_pred[0])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 64)             │        17,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,499 (72.26 KB)

 Trainable params: 18,499 (72.26 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
49/49 - 3s - 56ms/step - loss: 3.0003 - mse: 3.0003 - val_loss: 0.6165 - val_mse: 0.6165
Epoch 2/50
49/49 - 0s - 6ms/step - loss: 0.2023 - mse: 0.2023 - val_loss: 0.0405 - val_mse: 0.0405
Epoch 3/50
49/49 - 0s - 5ms/step - loss: 0.0760 - mse: 0.0760 - val_loss: 0.0264 - val_mse: 0.0264
Epoch 4/50
49/49 - 0s - 5ms/step - loss: 0.0632 - mse: 0.0632 - val_loss: 0.0236 - val_mse: 0.0236
Epoch 5/50
49/49 - 0s - 5ms/step - loss: 0.0647 - mse: 0.0647 - val_loss: 0.0232 - val_mse: 0.0232
Epoch 6/50
49/49 - 0s - 6ms/step - loss: 0.0600 - mse: 0.0600 - val_loss: 0.0223 - val_mse: 0.0223
Epoch 7/50
49/49 - 0s - 5ms/step - loss: 0.0570 - mse: 0.0570 - val_loss: 0.0211 - val_mse: 0.0211
Epoch 8/50
49/49 - 0s - 5ms/step - loss: 0.0598 - mse: 0.0598 - val_loss: 0.0220 - val_mse: 0.0220
Epoch 9/50
49/49 - 0s - 6ms/step - loss: 0.0545 - mse: 0.0545 - val_loss: 0.0209 - val_mse: 0.0209
Epoch 10/50
49/49 - 0s - 6ms/step - loss: 0.0515 - mse: 0.0515 - val_loss: 0.0192 - val_mse: 0.0192
Epoch 11

In [ ]:

# ————— تعریف مدل —————
model_backward = Sequential([
    # لایه LSTM: 4 واحد پنهان، فقط خروجی نهایی را باز می‌گرداند
    LSTM(64, input_shape=(2, 3), return_sequences=False),
    # برای کاهش بیش‌برازش، کمی Dropout
    Dropout(0.2),
    # یک لایه میانی برای افزایش ظرفیت غیرخطی مدل
    Dense(16, activation='relu'),
    # لایه خروجی با 3 نورون (برای هر feature یک مقدار)
    Dense(3)
])

model_backward.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mse']
)
model_backward.summary()

# ————— تنظیم EarlyStopping —————
es = EarlyStopping(
    monitor='val_loss',
    patience=50,
    restore_best_weights=True
)

# ————— آموزش مدل —————
history = model_backward.fit(
    rev_x_train, rev_y_train,
    epochs=50,
    batch_size=64,
    validation_split=0.2,
    # callbacks=[es],
    verbose=2
)

# ————— پیش‌بینی —————
# مثال: پیش‌بینی برای آخرین دو سطر
x_new = rev_x_train[-1][np.newaxis, ...]      # شکل (1,2,3)
y_pred = model_forward.predict(x_new)       # شکل (1,3)
print("پیش‌بینی:", y_pred[0])

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 64)             │        17,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,499 (72.26 KB)

 Trainable params: 18,499 (72.26 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
49/49 - 4s - 87ms/step - loss: 3.5171 - mse: 3.5171 - val_loss: 1.0150 - val_mse: 1.0150
Epoch 2/50
49/49 - 0s - 6ms/step - loss: 0.2405 - mse: 0.2405 - val_loss: 0.0536 - val_mse: 0.0536
Epoch 3/50
49/49 - 0s - 6ms/step - loss: 0.0796 - mse: 0.0796 - val_loss: 0.0293 - val_mse: 0.0293
Epoch 4/50
49/49 - 0s - 6ms/step - loss: 0.0627 - mse: 0.0627 - val_loss: 0.0255 - val_mse: 0.0255
Epoch 5/50
49/49 - 0s - 5ms/step - loss: 0.0642 - mse: 0.0642 - val_loss: 0.0232 - val_mse: 0.0232
Epoch 6/50
49/49 - 0s - 5ms/step - loss: 0.0582 - mse: 0.0582 - val_loss: 0.0210 - val_mse: 0.0210
Epoch 7/50
49/49 - 0s - 5ms/step - loss: 0.0497 - mse: 0.0497 - val_loss: 0.0205 - val_mse: 0.0205
Epoch 8/50
49/49 - 0s - 6ms/step - loss: 0.0489 - mse: 0.0489 - val_loss: 0.0184 - val_mse: 0.0184
Epoch 9/50
49/49 - 0s - 6ms/step - loss: 0.0510 - mse: 0.0510 - val_loss: 0.0175 - val_mse: 0.0175
Epoch 10/50
49/49 - 0s - 5ms/step - loss: 0.0464 - mse: 0.0464 - val_loss: 0.0157 - val_mse: 0.0157
Epoch 11

In [ ]:
Total_Feature = []
Total_Label_Feature = []

In [ ]:
TOTAL_ERROR_PLAIN_HEALTH = []
Total_Feature = []
Total_Label_Feature = []

window_size = 2
tail_size_config = 5  # اسمش رو عوض کردم تا با متغیر داخل حلقه اشتباه نشه

for picture in z_health[::4]:
    picture_reshaped = picture.reshape(16, 3)
    n_samples = picture_reshaped.shape[0] - window_size  # = 14

    if n_samples <= 0:
        continue

    # ساخت داده‌های ورودی/خروجی
    new_sample_x, new_sample_y = [], []
    for i in range(n_samples):
        new_sample_x.append(picture_reshaped[i : i + window_size])
        new_sample_y.append(picture_reshaped[i + window_size])

    new_sample_x_array = np.array(new_sample_x)  # (14, 2, 3)
    new_sample_y_array = np.array(new_sample_y)  # (14, 3)

    # پیش‌بینی کل توالی
    total_y_pre = []
    for seq in new_sample_x_array:
        t = np.expand_dims(seq, axis=0)  # (1, 2, 3)
        y_pre = model_forward.predict(t, verbose=0)
        total_y_pre.append(y_pre[0])
    total_y_pre = np.array(total_y_pre)  # (14, 3)

    # محاسبه خطا (MSE عنصری)
    diff_error = (total_y_pre - new_sample_y_array) ** 2  # (14, 3)
    TOTAL_ERROR_PLAIN_HEALTH.append(diff_error)

    # --- آمارهای کلی ---
    avg_error = np.mean(diff_error)
    mse = np.sum(diff_error)
    max_error = np.max(diff_error)
    std_error = np.std(diff_error)
    last_error = diff_error[-1].mean()  # تبدیل به اسکالر
    min_error = np.min(diff_error)

    # --- ویژگی‌های انتهایی ---
    n = len(diff_error)
    tail_size = min(tail_size_config, n)

    tail_mean = np.mean(diff_error[-tail_size:])
    tail_max = np.max(diff_error[-tail_size:])

    if tail_size > 1:
        tail_flat = diff_error[-tail_size:].flatten()
        x = np.arange(len(tail_flat))
        y = tail_flat
        slope = np.cov(x, y)[0, 1] / (np.var(x) + 1e-8)
        slope = max(slope, 0.0)
    else:
        slope = 0.0

    total_error = np.sum(diff_error) + 1e-8
    tail_error_sum = np.sum(diff_error[-tail_size:])
    tail_ratio = tail_error_sum / total_error

    # --- تعریف نهایی ویژگی‌ها ---
    features = {
        "mse": mse,
        "mean": avg_error,
        "max": max_error,
        "std": std_error,
        "last": last_error,
        "min": min_error,
        "tail_mean": tail_mean,
        "tail_max": tail_max,
        "tail_slope": slope,
        "tail_ratio": tail_ratio,
    }

    Total_Feature.append([
        features["mse"],
        features["mean"],
        features["max"],
        features["std"],
        features["last"],
        features["min"],
        features["tail_mean"],
        features["tail_max"],
        features["tail_slope"],
        features["tail_ratio"],
    ])

    Total_Label_Feature.append('0')

In [16]:
rev_TOTAL_ERROR_PLAIN_HEALTH = []
rev_Total_Feature = []
rev_Total_Label_Feature = []

window_size = 2
tail_size = 5  # تعداد فریم‌های انتهایی برای تحلیل

for picture in rev_z_health[::4]:
    picture_reshaped = picture.reshape(16, 3)
    n_samples = picture_reshaped.shape[0] - window_size  # = 14

    if n_samples <= 0:
        continue  # اگر تصویر خیلی کوچیک بود، ردش کن

    # --- ساخت داده‌های معکوس (backward) ---
    new_sample_x, new_sample_y = [], []
    # i از n_samples تا 1 (معکوس)
    for i in range(n_samples, 0, -1):
        start_idx = -(i + window_size)
        end_idx = -i if i != 0 else None
        x_seq = picture_reshaped[start_idx:end_idx]  # طول = window_size
        y_target = picture_reshaped[-i]              # یک فریم

        # بررسی اینکه x_seq واقعاً window_size فریم داشته باشه
        if x_seq.shape[0] == window_size:
            new_sample_x.append(x_seq)
            new_sample_y.append(y_target)

    if len(new_sample_x) == 0:
        continue

    new_sample_x_array = np.array(new_sample_x)  # (T, window_size, 3)
    new_sample_y_array = np.array(new_sample_y)  # (T, 3)

    # --- پیش‌بینی با مدل backward ---
    total_y_pre = []
    for seq in new_sample_x_array:
        t = np.expand_dims(seq, axis=0)  # (1, window_size, 3)
        y_pred = model_backward.predict(t, verbose=0)
        total_y_pre.append(y_pred[0])
    total_y_pre = np.array(total_y_pre)  # (T, 3)

    # --- محاسبه خطا (MSE عنصری) ---
    diff_error = (total_y_pre - new_sample_y_array) ** 2  # shape: (T, 3)
    rev_TOTAL_ERROR_PLAIN_HEALTH.append(diff_error)

    # --- استخراج ویژگی‌های پیشرفته ---
    T = diff_error.shape[0]
    actual_tail = min(tail_size, T)

    # آمارهای کلی
    avg_error = np.mean(diff_error)
    mse = np.sum(diff_error)
    max_error = np.max(diff_error)
    std_error = np.std(diff_error)
    last_error = diff_error[-1]  # آخرین فریم
    min_error = np.min(diff_error)

    # --- ویژگی‌های جدید: تمرکز روی انتهای توالی ---
    tail_errors = diff_error[-actual_tail:]  # آخرین `actual_tail` فریم
    tail_mean = np.mean(tail_errors)
    tail_max = np.max(tail_errors)
    tail_sum = np.sum(tail_errors)
    total_sum = np.sum(diff_error) + 1e-8
    tail_ratio = tail_sum / total_sum

    # شیب خطا در انتهای توالی (آیا خطا داره زیاد می‌شه؟)
    if actual_tail > 1:
        tail_flat = tail_errors.flatten()
        x = np.arange(len(tail_flat))
        y = tail_flat
        # محاسبه شیب بدون نیاز به کتابخونه
        slope = np.cov(x, y)[0, 1] / (np.var(x) + 1e-8)
        slope = max(slope, 0.0)  # فقط شیب مثبت مهمه
    else:
        slope = 0.0

    # --- ذخیره ویژگی‌ها ---
    rev_Total_Feature.append([
        mse,
        avg_error,
        max_error,
        std_error,
        last_error.mean(),  # یا last_error.max() اگر بخوای
        min_error,
        tail_mean,
        tail_max,
        tail_ratio,
        slope
    ])
    rev_Total_Label_Feature.append('0')  # '0' = سالم

In [17]:
# --- آماده‌سازی داده‌های خام ---
TOTAL_ERROR_PLAIN_ALPHA = []
total_alpha_x_y_z, label_alpha = [], []

for i in store_data_set:
    for img in store_data_set[i]:
        if i == 'picture_alpha':
            total_alpha_x_y_z.append(img)
            label_alpha.append(i)

x_alpha, y_alpha, z_alpha = [], [], []
for i in total_alpha_x_y_z:
    x_alpha.append(i['x'])
    y_alpha.append(i['y'])
    z_alpha.append(i['z'])

x_alpha = np.array(x_alpha).reshape(-1, 16, 3, 1)
y_alpha = np.array(y_alpha).reshape(-1, 16, 3, 1)
z_alpha = np.array(z_alpha).reshape(-1, 16, 3, 1)
z_alpha = np.where(z_alpha < 0, 0, z_alpha)

# --- پردازش هر تصویر ---
r = 0
window_size = 2  # مطمئن شو این تعریف شده (اگر قبلاً نیست، اینجا هست)

for picture in z_alpha[::4]:
    picture_reshaped = picture.reshape(16, 3)
    n_samples = picture_reshaped.shape[0] - window_size  # = 14

    if n_samples <= 0:
        continue

    # ساخت دنباله‌های ورودی/خروجی
    new_sample_x, new_sample_y = [], []
    for i in range(n_samples):
        new_sample_x.append(picture_reshaped[i : i + window_size])
        new_sample_y.append(picture_reshaped[i + window_size])

    new_sample_x_array = np.array(new_sample_x)  # (T, window_size, 3)
    new_sample_y_array = np.array(new_sample_y)  # (T, 3)

    # پیش‌بینی
    total_y_pre = []
    for seq in new_sample_x_array:
        t = np.expand_dims(seq, axis=0)
        y_pre = model_forward.predict(t, verbose=0)
        total_y_pre.append(y_pre[0])
    total_y_pre = np.array(total_y_pre)

    # محاسبه خطا (MSE عنصری)
    diff_error = (total_y_pre - new_sample_y_array) ** 2  # shape: (T, 3)
    TOTAL_ERROR_PLAIN_ALPHA.append(diff_error)

    # --- آمارهای کلی ---
    avg_error = np.mean(diff_error)
    mse = np.sum(diff_error)
    max_error = np.max(diff_error)
    std_error = np.std(diff_error)
    last_error = diff_error[-1].mean()  # ✅ تبدیل آرایه (3,) به عدد
    min_error = np.min(diff_error)

    # --- ویژگی‌های انتهایی (همانند داده‌های سالم) ---
    T = len(diff_error)
    tail_size = min(5, T)

    tail_mean = np.mean(diff_error[-tail_size:])
    tail_max = np.max(diff_error[-tail_size:])

    if tail_size > 1:
        tail_flat = diff_error[-tail_size:].flatten()
        x = np.arange(len(tail_flat))
        y = tail_flat
        slope = np.cov(x, y)[0, 1] / (np.var(x) + 1e-8)
        slope = max(slope, 0.0)  # فقط شیب مثبت
    else:
        slope = 0.0

    total_sum = np.sum(diff_error) + 1e-8
    tail_sum = np.sum(diff_error[-tail_size:])
    tail_ratio = tail_sum / total_sum

    # --- ذخیره ویژگی‌ها (دقیقاً مانند داده‌های سالم) ---
    Total_Feature.append([
        mse,            # 0
        avg_error,      # 1
        max_error,      # 2
        std_error,      # 3
        last_error,     # 4 ← عدد است، نه آرایه
        min_error,      # 5
        tail_mean,      # 6
        tail_max,       # 7
        slope,          # 8
        tail_ratio      # 9
    ])
    Total_Label_Feature.append('1')  # '1' = معیوب

    r += 1

print("تعداد تصاویر پردازش‌شده برای ALPHA:", r)

تعداد تصاویر پردازش‌شده برای ALPHA: 99


In [18]:
# --- آماده‌سازی داده‌های خام ---
rev_TOTAL_ERROR_PLAIN_ALPHA = []
rev_total_alpha_x_y_z, rev_label_alpha = [], []

for i in store_data_set:
    for img in store_data_set[i]:
        if i == 'picture_alpha':
            rev_total_alpha_x_y_z.append(img)
            rev_label_alpha.append(i)

rev_x_alpha, rev_y_alpha, rev_z_alpha = [], [], []
for i in rev_total_alpha_x_y_z:
    rev_x_alpha.append(i['x'])
    rev_y_alpha.append(i['y'])
    rev_z_alpha.append(i['z'])

rev_x_alpha = np.array(rev_x_alpha).reshape(-1, 16, 3, 1)
rev_y_alpha = np.array(rev_y_alpha).reshape(-1, 16, 3, 1)
rev_z_alpha = np.array(rev_z_alpha).reshape(-1, 16, 3, 1)
rev_z_alpha = np.where(rev_z_alpha < 0, 0, rev_z_alpha)

# --- پردازش معکوس ---
r = 0
window_size = 2  # مطمئن شو تعریف شده

for picture in rev_z_alpha[::4]:
    picture_reshaped = picture.reshape(16, 3)
    n_samples = picture_reshaped.shape[0] - window_size  # = 14

    if n_samples <= 0:
        continue

    # --- ساخت دنباله‌های معکوس ---
    new_sample_x, new_sample_y = [], []
    # i از n_samples تا 1 (معکوس)
    for i in range(n_samples, 0, -1):
        start_idx = -(i + window_size)
        end_idx = -i if i != 0 else None
        x_seq = picture_reshaped[start_idx:end_idx]
        y_target = picture_reshaped[-i]

        # بررسی طول x_seq
        if x_seq.shape[0] == window_size:
            new_sample_x.append(x_seq)
            new_sample_y.append(y_target)

    if len(new_sample_x) == 0:
        continue

    new_sample_x_array = np.array(new_sample_x)  # (T, window_size, 3)
    new_sample_y_array = np.array(new_sample_y)  # (T, 3)

    # --- پیش‌بینی با مدل backward ---
    total_y_pre = []
    for seq in new_sample_x_array:
        t = np.expand_dims(seq, axis=0)
        y_pre = model_backward.predict(t, verbose=0)
        total_y_pre.append(y_pre[0])
    total_y_pre = np.array(total_y_pre)

    # --- محاسبه خطا ---
    diff_error = (total_y_pre - new_sample_y_array) ** 2  # (T, 3)
    rev_TOTAL_ERROR_PLAIN_ALPHA.append(diff_error)

    # --- آمارهای کلی ---
    avg_error = np.mean(diff_error)
    mse = np.sum(diff_error)
    max_error = np.max(diff_error)
    std_error = np.std(diff_error)
    last_error = diff_error[-1].mean()  # ✅ تبدیل به عدد
    min_error = np.min(diff_error)

    # --- ویژگی‌های انتهایی (همانند forward) ---
    T = len(diff_error)
    tail_size = min(5, T)

    tail_mean = np.mean(diff_error[-tail_size:])
    tail_max = np.max(diff_error[-tail_size:])

    if tail_size > 1:
        tail_flat = diff_error[-tail_size:].flatten()
        x = np.arange(len(tail_flat))
        y = tail_flat
        slope = np.cov(x, y)[0, 1] / (np.var(x) + 1e-8)
        slope = max(slope, 0.0)
    else:
        slope = 0.0

    total_sum = np.sum(diff_error) + 1e-8
    tail_sum = np.sum(diff_error[-tail_size:])
    tail_ratio = tail_sum / total_sum

    # --- ذخیره ویژگی‌ها (10 ویژگی — همانند forward) ---
    rev_Total_Feature.append([
        mse,            # 0
        avg_error,      # 1
        max_error,      # 2
        std_error,      # 3
        last_error,     # 4 ← عدد است
        min_error,      # 5
        tail_mean,      # 6
        tail_max,       # 7
        slope,          # 8
        tail_ratio      # 9
    ])
    rev_Total_Label_Feature.append('1')  # '1' = معیوب

    r += 1

print("تعداد تصاویر پردازش‌شده (backward ALPHA):", r)

تعداد تصاویر پردازش‌شده (backward ALPHA): 99


In [19]:
TOTAL_ERROR_PLAIN_VOID = []
total_void_x_y_z, label_void = [], []

for i in store_data_set:
    for img in store_data_set[i]:
        if i == 'picture_void':
            total_void_x_y_z.append(img)
            label_void.append(i)

x_void, y_void, z_void = [], [], []
for i in total_void_x_y_z:
    x_void.append(i['x'])
    y_void.append(i['y'])
    z_void.append(i['z'])

x_void = np.array(x_void).reshape(-1, 16, 3, 1)
y_void = np.array(y_void).reshape(-1, 16, 3, 1)
z_void = np.array(z_void).reshape(-1, 16, 3, 1)
z_void = np.where(z_void < 0, 0, z_void)

r = 0
window_size = 2  # مطمئن شو تعریف شده

for picture in z_void[::4]:
    picture_reshaped = picture.reshape(16, 3)
    n_samples = picture_reshaped.shape[0] - window_size  # = 14

    if n_samples <= 0:
        continue

    new_sample_x, new_sample_y = [], []
    for i in range(n_samples):
        new_sample_x.append(picture_reshaped[i : i + window_size])
        new_sample_y.append(picture_reshaped[i + window_size])

    new_sample_x_array = np.array(new_sample_x)
    new_sample_y_array = np.array(new_sample_y)

    total_y_pre = []
    for seq in new_sample_x_array:
        t = np.expand_dims(seq, axis=0)
        y_pre = model_forward.predict(t, verbose=0)
        total_y_pre.append(y_pre[0])
    total_y_pre = np.array(total_y_pre)

    diff_error = (total_y_pre - new_sample_y_array) ** 2
    TOTAL_ERROR_PLAIN_VOID.append(diff_error)

    # --- آمارهای کلی ---
    avg_error = np.mean(diff_error)  # ✅ اضافه شد!
    mse = np.sum(diff_error)
    max_error = np.max(diff_error)
    std_error = np.std(diff_error)
    last_error = diff_error[-1].mean()  # ✅ تبدیل به عدد
    min_error = np.min(diff_error)

    # --- ویژگی‌های انتهایی (همانند سالم و ALPHA) ---
    T = len(diff_error)
    tail_size = min(5, T)

    tail_mean = np.mean(diff_error[-tail_size:])
    tail_max = np.max(diff_error[-tail_size:])

    if tail_size > 1:
        tail_flat = diff_error[-tail_size:].flatten()
        x = np.arange(len(tail_flat))
        y = tail_flat
        slope = np.cov(x, y)[0, 1] / (np.var(x) + 1e-8)
        slope = max(slope, 0.0)
    else:
        slope = 0.0

    total_sum = np.sum(diff_error) + 1e-8
    tail_sum = np.sum(diff_error[-tail_size:])
    tail_ratio = tail_sum / total_sum

    # --- ذخیره 10 ویژگی ---
    Total_Feature.append([
        mse,            # 0
        avg_error,      # 1 ← ✅ اضافه شد
        max_error,      # 2
        std_error,      # 3
        last_error,     # 4 ← عدد است
        min_error,      # 5
        tail_mean,      # 6
        tail_max,       # 7
        slope,          # 8
        tail_ratio      # 9
    ])
    Total_Label_Feature.append('2')  # '2' = VOID

    r += 1

print("تعداد تصاویر پردازش‌شده برای VOID:", r)

تعداد تصاویر پردازش‌شده برای VOID: 99


In [20]:
rev_TOTAL_ERROR_PLAIN_VOID = []
rev_total_void_x_y_z, rev_label_void = [], []

for i in store_data_set:
    for img in store_data_set[i]:
        if i == 'picture_void':
            rev_total_void_x_y_z.append(img)
            rev_label_void.append(i)

rev_x_void, rev_y_void, rev_z_void = [], [], []
for i in rev_total_void_x_y_z:
    rev_x_void.append(i['x'])
    rev_y_void.append(i['y'])
    rev_z_void.append(i['z'])

rev_x_void = np.array(rev_x_void).reshape(-1, 16, 3, 1)
rev_y_void = np.array(rev_y_void).reshape(-1, 16, 3, 1)
rev_z_void = np.array(rev_z_void).reshape(-1, 16, 3, 1)
rev_z_void = np.where(rev_z_void < 0, 0, rev_z_void)

r = 0
window_size = 2  # مطمئن شو تعریف شده

for picture in rev_z_void[::4]:
    picture_reshaped = picture.reshape(16, 3)
    n_samples = picture_reshaped.shape[0] - window_size  # = 14

    if n_samples <= 0:
        continue

    # --- ساخت دنباله‌های معکوس ---
    new_sample_x, new_sample_y = [], []
    for i in range(n_samples, 0, -1):
        start_idx = -(i + window_size)
        end_idx = -i if i != 0 else None
        x_seq = picture_reshaped[start_idx:end_idx]
        y_target = picture_reshaped[-i]

        if x_seq.shape[0] == window_size:
            new_sample_x.append(x_seq)
            new_sample_y.append(y_target)

    if len(new_sample_x) == 0:
        continue

    new_sample_x_array = np.array(new_sample_x)  # (T, window_size, 3)
    new_sample_y_array = np.array(new_sample_y)  # (T, 3)

    # --- پیش‌بینی با مدل BACKWARD (نه forward!) ---
    total_y_pre = []
    for seq in new_sample_x_array:
        t = np.expand_dims(seq, axis=0)
        y_pre = model_backward.predict(t, verbose=0)  # ✅ تغییر به model_backward
        total_y_pre.append(y_pre[0])
    total_y_pre = np.array(total_y_pre)

    # --- محاسبه خطا ---
    diff_error = (total_y_pre - new_sample_y_array) ** 2
    rev_TOTAL_ERROR_PLAIN_VOID.append(diff_error)

    # --- آمارهای کلی ---
    avg_error = np.mean(diff_error)  # ✅ اضافه شد
    mse = np.sum(diff_error)
    max_error = np.max(diff_error)
    std_error = np.std(diff_error)
    last_error = diff_error[-1].mean()  # ✅ تبدیل به عدد
    min_error = np.min(diff_error)

    # --- ویژگی‌های انتهایی ---
    T = len(diff_error)
    tail_size = min(5, T)

    tail_mean = np.mean(diff_error[-tail_size:])
    tail_max = np.max(diff_error[-tail_size:])

    if tail_size > 1:
        tail_flat = diff_error[-tail_size:].flatten()
        x = np.arange(len(tail_flat))
        y = tail_flat
        slope = np.cov(x, y)[0, 1] / (np.var(x) + 1e-8)
        slope = max(slope, 0.0)
    else:
        slope = 0.0

    total_sum = np.sum(diff_error) + 1e-8
    tail_sum = np.sum(diff_error[-tail_size:])
    tail_ratio = tail_sum / total_sum

    # --- ذخیره 10 ویژگی (همانند forward) ---
    rev_Total_Feature.append([
        mse,            # 0
        avg_error,      # 1
        max_error,      # 2
        std_error,      # 3
        last_error,     # 4 ← عدد است
        min_error,      # 5
        tail_mean,      # 6
        tail_max,       # 7
        slope,          # 8
        tail_ratio      # 9
    ])
    rev_Total_Label_Feature.append('2')  # '2' = VOID

    r += 1

print("تعداد تصاویر پردازش‌شده (backward VOID):", r)

تعداد تصاویر پردازش‌شده (backward VOID): 99


In [21]:
TOTAL_ERROR_PLAIN_VOID_ALPHA = []
total_void_alpha_x_y_z, label_void_alpha = [], []

for i in store_data_set:
    for img in store_data_set[i]:
        if i == 'picture_void_alpha':
            total_void_alpha_x_y_z.append(img)
            label_void_alpha.append(i)

x_void_alpha, y_void_alpha, z_void_alpha = [], [], []
for i in total_void_alpha_x_y_z:
    x_void_alpha.append(i['x'])
    y_void_alpha.append(i['y'])
    z_void_alpha.append(i['z'])

x_void_alpha = np.array(x_void_alpha).reshape(-1, 16, 3, 1)
y_void_alpha = np.array(y_void_alpha).reshape(-1, 16, 3, 1)
z_void_alpha = np.array(z_void_alpha).reshape(-1, 16, 3, 1)
z_void_alpha = np.where(z_void_alpha < 0, 0, z_void_alpha)

r = 0
window_size = 2  # مطمئن شو تعریف شده

for picture in z_void_alpha[::4]:  # هر 4 تا یکی
    picture_reshaped = picture.reshape(16, 3)
    n_samples = picture_reshaped.shape[0] - window_size  # = 14

    if n_samples <= 0:
        continue

    new_sample_x, new_sample_y = [], []
    for i in range(n_samples):
        new_sample_x.append(picture_reshaped[i : i + window_size])
        new_sample_y.append(picture_reshaped[i + window_size])

    new_sample_x_array = np.array(new_sample_x)
    new_sample_y_array = np.array(new_sample_y)

    total_y_pre = []
    for seq in new_sample_x_array:
        t = np.expand_dims(seq, axis=0)
        y_pre = model_forward.predict(t, verbose=0)
        total_y_pre.append(y_pre[0])
    total_y_pre = np.array(total_y_pre)

    diff_error = (total_y_pre - new_sample_y_array) ** 2
    TOTAL_ERROR_PLAIN_VOID_ALPHA.append(diff_error)

    # --- آمارهای کلی ---
    avg_error = np.mean(diff_error)  # ✅ اضافه شد
    mse = np.sum(diff_error)
    max_error = np.max(diff_error)
    std_error = np.std(diff_error)
    last_error = diff_error[-1].mean()  # ✅ تبدیل به عدد
    min_error = np.min(diff_error)

    # --- ویژگی‌های انتهایی ---
    T = len(diff_error)
    tail_size = min(5, T)

    tail_mean = np.mean(diff_error[-tail_size:])
    tail_max = np.max(diff_error[-tail_size:])

    if tail_size > 1:
        tail_flat = diff_error[-tail_size:].flatten()
        x = np.arange(len(tail_flat))
        y = tail_flat
        slope = np.cov(x, y)[0, 1] / (np.var(x) + 1e-8)
        slope = max(slope, 0.0)
    else:
        slope = 0.0

    total_sum = np.sum(diff_error) + 1e-8
    tail_sum = np.sum(diff_error[-tail_size:])
    tail_ratio = tail_sum / total_sum

    # --- ذخیره 10 ویژگی ---
    Total_Feature.append([
        mse,            # 0
        avg_error,      # 1 ← ✅ اضافه شد
        max_error,      # 2
        std_error,      # 3
        last_error,     # 4 ← عدد است
        min_error,      # 5
        tail_mean,      # 6
        tail_max,       # 7
        slope,          # 8
        tail_ratio      # 9
    ])
    Total_Label_Feature.append('3')  # '3' = VOID_ALPHA

    r += 1
    # if r == 200: break  # فقط اگر واقعاً نیاز داری فعالش کن

print("تعداد تصاویر پردازش‌شده برای VOID_ALPHA:", r)

تعداد تصاویر پردازش‌شده برای VOID_ALPHA: 99


In [22]:
rev_TOTAL_ERROR_PLAIN_VOID_ALPHA = []
rev_total_void_alpha_x_y_z, rev_label_void_alpha = [], []

# --- جمع‌آوری داده‌های خام ---
for i in store_data_set:
    for img in store_data_set[i]:
        if i == 'picture_void_alpha':
            rev_total_void_alpha_x_y_z.append(img)
            rev_label_void_alpha.append(i)

rev_x_void_alpha, rev_y_void_alpha, rev_z_void_alpha = [], [], []

# ❗ تغییر کلیدی: از rev_total_void_alpha_x_y_z استفاده کن، نه total_void_alpha_x_y_z
for i in rev_total_void_alpha_x_y_z:  # ✅ اینجا اصلاح شد
    rev_x_void_alpha.append(i['x'])
    rev_y_void_alpha.append(i['y'])
    rev_z_void_alpha.append(i['z'])

rev_x_void_alpha = np.array(rev_x_void_alpha).reshape(-1, 16, 3, 1)
rev_y_void_alpha = np.array(rev_y_void_alpha).reshape(-1, 16, 3, 1)
rev_z_void_alpha = np.array(rev_z_void_alpha).reshape(-1, 16, 3, 1)
rev_z_void_alpha = np.where(rev_z_void_alpha < 0, 0, rev_z_void_alpha)

# --- پردازش معکوس ---
r = 0
window_size = 2  # مطمئن شو تعریف شده

for picture in rev_z_void_alpha[::4]:
    picture_reshaped = picture.reshape(16, 3)
    n_samples = picture_reshaped.shape[0] - window_size  # = 14

    if n_samples <= 0:
        continue

    new_sample_x, new_sample_y = [], []
    # --- حلقه معکوس ایمن ---
    for i in range(n_samples, 0, -1):
        start_idx = -(i + window_size)
        end_idx = -i if i != 0 else None
        x_seq = picture_reshaped[start_idx:end_idx]
        y_target = picture_reshaped[-i]

        if x_seq.shape[0] == window_size:
            new_sample_x.append(x_seq)
            new_sample_y.append(y_target)

    if len(new_sample_x) == 0:
        continue

    new_sample_x_array = np.array(new_sample_x)
    new_sample_y_array = np.array(new_sample_y)

    # --- پیش‌بینی با مدل BACKWARD ---
    total_y_pre = []
    for seq in new_sample_x_array:
        t = np.expand_dims(seq, axis=0)
        y_pre = model_backward.predict(t, verbose=0)  # ✅ model_backward
        total_y_pre.append(y_pre[0])
    total_y_pre = np.array(total_y_pre)

    # --- محاسبه خطا ---
    diff_error = (total_y_pre - new_sample_y_array) ** 2
    rev_TOTAL_ERROR_PLAIN_VOID_ALPHA.append(diff_error)

    # --- آمارهای کلی ---
    avg_error = np.mean(diff_error)  # ✅ اضافه شد
    mse = np.sum(diff_error)
    max_error = np.max(diff_error)
    std_error = np.std(diff_error)
    last_error = diff_error[-1].mean()  # ✅ تبدیل به عدد
    min_error = np.min(diff_error)

    # --- ویژگی‌های انتهایی ---
    T = len(diff_error)
    tail_size = min(5, T)

    tail_mean = np.mean(diff_error[-tail_size:])
    tail_max = np.max(diff_error[-tail_size:])

    if tail_size > 1:
        tail_flat = diff_error[-tail_size:].flatten()
        x = np.arange(len(tail_flat))
        y = tail_flat
        slope = np.cov(x, y)[0, 1] / (np.var(x) + 1e-8)
        slope = max(slope, 0.0)
    else:
        slope = 0.0

    total_sum = np.sum(diff_error) + 1e-8
    tail_sum = np.sum(diff_error[-tail_size:])
    tail_ratio = tail_sum / total_sum

    # --- ذخیره 10 ویژگی ---
    rev_Total_Feature.append([
        mse,            # 0
        avg_error,      # 1
        max_error,      # 2
        std_error,      # 3
        last_error,     # 4 ← عدد است
        min_error,      # 5
        tail_mean,      # 6
        tail_max,       # 7
        slope,          # 8
        tail_ratio      # 9
    ])
    rev_Total_Label_Feature.append('3')  # '3' = VOID_ALPHA

    r += 1
    # if r == 200: break  # فقط اگر واقعاً نیاز داری

print("تعداد تصاویر پردازش‌شده (backward VOID_ALPHA):", r)

تعداد تصاویر پردازش‌شده (backward VOID_ALPHA): 99


In [23]:
X = np.array(Total_Feature + rev_Total_Feature)
y = np.array(Total_Label_Feature + rev_Total_Label_Feature)

In [24]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100)
model.fit(X, y)

RandomForestClassifier()

In [26]:
# تقسیم 80% آموزش، 20% تست
X_train_end, X_test_end, y_train_end, y_test_end = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [27]:
# ساخت و آموزش مدل
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_end, y_train_end)

RandomForestClassifier(random_state=42)

In [28]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [30]:
# پیش‌بینی روی داده‌های تست
y_pred = model.predict(X_test_end)

# چاپ گزارش کامل
print("\n" + "="*60)
print("📊 گزارش طبقه‌بندی")
print("="*60)
print(classification_report(y_test_end, y_pred, target_names=[
    'سالم (0)', 'ALPHA (1)', 'VOID (2)', 'VOID+ALPHA (3)'
]))


📊 گزارش طبقه‌بندی
                precision    recall  f1-score   support

      سالم (0)       0.70      0.57      0.63        40
     ALPHA (1)       0.36      0.44      0.40        39
      VOID (2)       0.30      0.33      0.31        40
VOID+ALPHA (3)       0.40      0.35      0.37        40

      accuracy                           0.42       159
     macro avg       0.44      0.42      0.43       159
  weighted avg       0.44      0.42      0.43       159

